In [ ]:
import pandas as pd
import pypsa
%matplotlib inline
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import cartopy.crs as ccrs
import ipynb
import os
from os import walk
import plotly.express as px
import json
import plotly.graph_objs as gobj
import plotly.graph_objects as go

In [ ]:
def load_data(mypath1,filenames):
    networks = {}
    planning_horizon = []
    for file in filenames:
        year = file[-7:-3]
        print("Loading year {} ".format(year))
        networks[year] = pypsa.Network(os.path.join(mypath1,file))
        planning_horizon.append(year)
    print("I have done my deed now set me Free, mortal!")
    return(networks, planning_horizon)

In [ ]:
def get_grid_df(n,carrier):
    """Gets all capacity dataframes for the grid
    Parameters
    ----------
    n : Networks
    carrier : string (== "gas pipeline", "AC", "H2 pipeline")
        carrier (e.g. "gas pipeline", "DC") used in lines and links for transporting a specific energy
    
    Returns
    -------
    two dataframes:
    normal = classic grid, including newly build H2 grid
    retro only used for H2 retrofitted piplines.
    """
    normal = n.copy()
    retro = n.copy()
    dc = n.copy()
    if carrier == 'AC':
        normal.lines = n.lines[n.lines.type =="Al/St 240/40 4-bundle 380.0"]
        dc.links = n.links[n.links.carrier == "DC"]
        for bus in normal.buses[n.buses.carrier=="AC"].index:
            dc.buses.loc[f"{bus}", ["x","y"]] = normal.buses.loc[bus, ["x","y"]]
    else: 
        normal.links = n.links[(n.links.carrier == carrier)]
        carrier_retro = carrier + " retrofit"
        retro.links = n.links[(n.links.carrier == carrier_retro)]
        for bus in normal.buses[n.buses.carrier=="AC"].index:
            normal.buses.loc[f"{bus}"+ " "  + carrier[:3].strip(), ["x","y"]] = normal.buses.loc[bus, ["x","y"]]
            retro.buses.loc[f"{bus}"+ " " + carrier[:3].strip(), ["x","y"]] = retro.buses.loc[bus, ["x","y"]]
    return(normal,retro,dc)

In [ ]:
countries = gpd.read_file(r'C:\Users\johannes.misensky\OneDrive - AGGM\Dokumente\pyp\Shapefile\shape_nuts2.shp')
#countries.plot(figsize = (15, 12))

In [ ]:
mypath = r"C:\Users\johannes.misensky\OneDrive - AGGM\Dokumente\ESM_Aus\postnetworks\test_1"
mypath1 = f"{mypath}"
filenames = next(walk(mypath1), (None, None, []))[2]

networks = {}
planning_horizon = []
filenames

networks, planning_horizon = load_data(mypath1,filenames)

In [ ]:
#countries.to_file("path_to_GeoJSON _file", driver = "GeoJSON")
with open("path_to_GeoJSON _file") as geofile:
    j_file = json.load(geofile)

In [ ]:
# for i in range(100):
#     print(j_file['features'][i]['properties'])

# #j_file['features'][10]['properties']


#normal.links[normal.links.index.str.contains('AT')]

In [ ]:
n= networks['2015']

carrier = "AC"

normal, retro, dc = get_grid_df(n,carrier)

In [ ]:
dc.buses



In [ ]:
# carrier= carrier[:3].strip()          #dieser Snip ist wsl nicht notwendig => sollte später entfernt werden wenn get_data_skript verbessert ist. 
# df = normal.buses[normal.buses.carrier == carrier].copy()
# df['ct'] = df.index.str[:2]

# fig = go.Figure(data=go.Choropleth(
#     geojson=j_file,
#     featureidkey= 'properties.COUNTRY_ID',
#     locationmode='geojson-id',
#     locations=df['ct'],
#     z=df['v_nom'], #Platzhalter für später = Soll eventuell mal verbrauch werden 
#     autocolorscale=False,
#     showscale=False,
#     hoverinfo = 'text',
#     text = df['ct']
    
# ))


# if carrier == 'AC':
    
#     for i in range(len(normal.lines.s_nom_opt)):
#         fig.add_trace(
#             go.Scattergeo(
#                 locationmode = 'geojson-id',
#                 lon = [df.loc[normal.lines.bus0[i]]['x'].round(3), df.loc[normal.lines.bus1[i]]['x'].round(3)],
#                 lat = [df.loc[normal.lines.bus0[i]]['y'].round(3), df.loc[normal.lines.bus1[i]]['y'].round(3)],
#                 mode = 'lines',
#                 line = dict(width = (data['s_nom_opt'][i])/1e4,color = 'green'),
#                 # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
#             )
#         )
# else:
    
#     for i in range(len(normal.links.p_nom_opt)):
#         fig.add_trace(
#             go.Scattergeo(
#                 locationmode = 'geojson-id',
#                 lon = [df.loc[normal.links.bus0[i]]['x'].round(3), df.loc[normal.links.bus1[i]]['x'].round(3)],
#                 lat = [df.loc[normal.links.bus0[i]]['y'].round(3), df.loc[normal.links.bus1[i]]['y'].round(3)],
#                 mode = 'lines',
#                 line = dict(width = (data['p_nom_opt'][i])/1e4,color = 'red'),
#                 # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
#             )
#         )
#     for i in range(len(retro.links.p_nom_opt)):
#         fig.add_trace(
#             go.Scattergeo(
#                 locationmode = 'geojson-id',
#                 lon = [df.loc[retro.links.bus0[i]]['x'].round(3), df.loc[retro.links.bus1[i]]['x'].round(3)],
#                 lat = [df.loc[retro.links.bus0[i]]['y'].round(3), df.loc[retro.links.bus1[i]]['y'].round(3)],
#                 mode = 'lines',
#                 line = dict(width = (data['p_nom_opt_retro'][i])/1e4,color = 'blue'),
#                 hoverinfo = 'text',
#                 text = data['p_nom_opt_retro'][i],
#                 # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
#             )
#         )


# fig.update_geos(
#     visible=False, resolution=50, scope="europe",
#     showcountries=True, countrycolor="Black",
#     #showsubunits=True, subunitcolor="Blue"
# )



# fig.update_layout(
#     autosize=False,
#     width=1000,
#     height=1000,
#     showlegend = False,
# )
# fig.show()

In [ ]:
def plot_grid(normal, retro, carrier, year):
    """Plots a Geo. Figure in Plotly containing all clusters and a given grid
    Parameters
    ----------
    normal : Dataframe containing Links and Buses for AC and H2/gas grid, should be provided by get_grid_df funktion
    retro : Dataframe containing Links and Buses for retrofitted H2 grid, should be provided by get_grid_df funktion
    carrier : string (== "gas pipeline", "AC", "H2 pipeline")
        carrier (e.g. "gas pipeline", "AC") used to differenciate what grid to plot
    
    Returns
    -------
    Grid plot of a given carrier
    """
    carrier= carrier[:3].strip()          #this part is unnecessary here should be deleted as soon as get_grid_df is modified 
    df = normal.buses[normal.buses.carrier == carrier].copy()
    df['ct'] = df.index.str[:5]
    
    fig = go.Figure(data=go.Choropleth(
        #eojson=j_file,
        featureidkey= 'properties.COUNTRY_ID',
        locationmode='geojson-id', # honestly no idea what this does
        locations=df['ct'],
        z=df['v_nom'], # Placeholder Variable for a later use could be colorcode for powerussage?  
        autocolorscale=False,
        showscale=False,
        hoverinfo = 'text',
        text = df['ct']

    ))

    #Plot grid lines and set width, AC has to be treated specially 
    if carrier == 'AC':

        for i in range(len(normal.lines.s_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    locationmode = 'geojson-id',# honestly no idea what this does
                    lon = [df.loc[normal.lines.bus0[i]]['x'].round(3), df.loc[normal.lines.bus1[i]]['x'].round(3)],
                    lat = [df.loc[normal.lines.bus0[i]]['y'].round(3), df.loc[normal.lines.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (normal.lines.s_nom_opt[i])/1e4,color = 'green'),
                    # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),  => Could be used later
                    name="AC Grid"
                )
            )
        for i in range(len(dc.links.p_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    locationmode = 'geojson-id',# honestly no idea what this does
                    lon = [df.loc[retro.links.bus0[i]]['x'].round(3), df.loc[retro.links.bus1[i]]['x'].round(3)],
                    lat = [df.loc[retro.links.bus0[i]]['y'].round(3), df.loc[retro.links.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (dc.links.p_nom_opt[i])/1e4,color = 'red'),
                    hoverinfo = 'text',
                    text = dc.links.p_nom[i].round(0),
                    # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
                    name="DC Grid"
                )
            )

    else:

        for i in range(len(normal.links.p_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    locationmode = 'geojson-id',# honestly no idea what this does
                    lon = [df.loc[normal.links.bus0[i]]['x'].round(3), df.loc[normal.links.bus1[i]]['x'].round(3)],
                    lat = [df.loc[normal.links.bus0[i]]['y'].round(3), df.loc[normal.links.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (normal.links.p_nom_opt[i])/1e4,color = 'red'),
                    # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
                    text = normal.links.p_nom[i],
                    name="new Pipeline"
                    
                )
            )
        for i in range(len(retro.links.p_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    locationmode = 'geojson-id',# honestly no idea what this does
                    lon = [df.loc[retro.links.bus0[i]]['x'].round(3), df.loc[retro.links.bus1[i]]['x'].round(3)],
                    lat = [df.loc[retro.links.bus0[i]]['y'].round(3), df.loc[retro.links.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (retro.links.p_nom_opt[i])/1e4,color = 'blue'),
                    hoverinfo = 'text',
                    text = retro.links.p_nom[i].round(0),
                    name="retrofitted Pipeline"
                    # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
                )
            )

    #centralize on europe
    fig.update_geos(
        visible=False, resolution=50, 
        scope="europe",
        # center=dict(lon=11, lat=40),
        # projection_rotation=dict(lon=30, lat=30, roll=30),
        # lataxis_range=[10,11], lonaxis_range=[33,41],
        showcountries=True, countrycolor="Black",
        #showsubunits=True, subunitcolor="Blue"
    )
    

    #Scale Figure 
    fig.update_layout(
        title=f" Grid Capacity in {year}",
        autosize=False,
        width=1000,
        height=1000,
        showlegend = True,
        legend=dict(itemsizing='constant')
    )
    names = set()
    fig.for_each_trace(
        lambda trace:
        trace.update(showlegend=False)
        if (trace.name in names) else names.add(trace.name))

    return(fig)

In [ ]:
pip install dash-bootstrap-components

In [ ]:
from dash import Dash, dcc, Output, Input  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import plotly.express as px

# incorporate data into app
#df = px.data.medals_long()

# Build your components
app = Dash(__name__, external_stylesheets=[dbc.themes.SOLAR])
mytitle = dcc.Markdown(children='# TEST TEST')
mygraph = dcc.Graph(figure={})
dropdown = dcc.Dropdown(options=['H2 pipeline', 'gas pipeline', "AC"],
                        value='H2 pipeline',  # initial value displayed when page first loads
                        clearable=False)
dropdown_year = dcc.Dropdown(options=['2015','2020','2030','2040','2050'],
                        value='2015',  # initial value displayed when page first loads
                        clearable=False)

# Customize your own Layout
app.layout = dbc.Container([mytitle, dropdown, dropdown_year, mygraph])

# Callback allows components to interact
@app.callback(
    Output(mygraph, component_property='figure'),
    Input(dropdown, component_property='value'),
    Input(dropdown_year, component_property='value')
)
def update_graph(user_input, dropdown_year):  # function arguments come from the component property of the Input
    carrier = user_input
    n = networks[dropdown_year]
    normal,retro,dc = get_grid_df(n,carrier)
    fig = plot_grid(normal, retro, carrier, dropdown_year)
    return fig  # returned objects are assigned to the component property of the Output




In [ ]:
# Run app
if __name__=='__main__':
    app.run_server(port=8043)

## Sachen die noch fehlen:

1) Österreich richtig plotten
2) Legende
3) optimieren des Plots
4) Flussrichtung

In [ ]:
dict_t = {}
years = ['2015', '2020', '2030', '2040', '2050']

for year in years:

    n= networks[year]

    carrier = "gas pipeline"

    normal,retro = get_grid_df(n,carrier)
    
    dict_t[year] = normal.links[normal.links.carrier == 'gas pipeline'][['bus0','bus1','carrier','p_nom','p_nom_opt']]


In [ ]:
normal.links

In [ ]:
dict_t['2020']


In [ ]:
dict_t['2030']

In [ ]:
dict_t['2040']

In [ ]:
dict_t['2050']

In [ ]:
n = networks['2050']

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattergeo())
fig.update_geos(
    center=dict(lon=11, lat=40),
    projection_rotation=dict(lon=30, lat=30, roll=30),
    lataxis_range=[10,12], lonaxis_range=[20, 41]
)
fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
normal.buses[:11]

In [ ]:
at = ['AT0 0', 'AT0 1', 'AT0 2', 'AT0 3', 'AT0 4', 'AT0 5', 'AT0 6', 'AT0 7', 'AT0 8', 'AT0 9'] 

In [ ]:
# Build your components
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SOLAR])
mytext = dcc.Markdown(children='')
myinput = dbc.Input(value="# Hello World - let's build web apps in Python!")
app.title = "Avocado Analytics: Understand Your Avocados!"

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.P(children="🥑", className="header-emoji"),
                html.H1(
                    children="Avocado Analytics", className="header-title"
                ),
                html.P(
                    children="Analyze the behavior of avocado prices"
                    " and the number of avocados sold in the US"
                    " between 2015 and 2018",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
    dcc.Graph(figure=fig)
            )
# Customize your own Layout
#app.layout = dbc.Container([mytext, myinput])

# Callback allows components to interact
@app.callback(
    Output(mytext, component_property='children'),
    Input(myinput, component_property='value')
)
def update_title(user_input):  # function arguments come from the component property of the Input
    return user_input  # returned objects are assigned to the component property of the Output

In [ ]:
normal.links.head()

In [ ]:
len(df)

In [ ]:
normal.buses[normal.buses.carrier == "gas"]

In [ ]:
normal.links.p_nom_opt[0]

# AB hier beginnt BS 

In [ ]:
n.lines.s_nom_opt

In [ ]:
#  i=1
# for feature in j_file["features"]:
#     feature ['id'] = str(i).zfill(2)
#     i += 1
# normal.lines = n.lines[n.lines.type =="Al/St 240/40 4-bundle 380.0"]

In [ ]:
#normal.buses[normal.buses.carrier == 'AC'].head()

In [ ]:
#normal.lines[normal.lines.type =="Al/St 240/40 4-bundle 380.0"].head()

In [ ]:
carrier= carrier[:3].strip()
df = normal.buses[normal.buses.carrier == carrier].copy()
df2 = normal.lines[normal.lines.type =="Al/St 240/40 4-bundle 380.0"].copy()
df['ct'] = df.index.str[:2]
coord_round=3


In [ ]:
data = {"index" : list(normal.links.index),
            "bus0" : list(normal.links.bus0),
            "bus1" : list(normal.links.bus1),
            "x0" : list(normal.links.bus0.map(df.x).round(coord_round)),
            "y0" : list(normal.links.bus0.map(df.y).round(coord_round)),
            "x1" : list(normal.links.bus1.map(df.x).round(coord_round)),
            "y1" : list(normal.links.bus1.map(df.y).round(coord_round)),
            "p_nom_opt" : list(normal.links.p_nom_opt.round(0)),
            "s_nom_opt" : list(normal.lines.s_nom_opt.round(0)),
            "x0_AC" : list(normal.lines.bus0.map(df.x).round(coord_round)),
            "y0_AC" : list(normal.lines.bus0.map(df.y).round(coord_round)),
            "x1_AC" : list(normal.lines.bus1.map(df.x).round(coord_round)),
            "y1_AC" : list(normal.lines.bus1.map(df.y).round(coord_round)),
        
        
        "index_retro" : list(retro.links.index),
            "bus0_retro" : list(retro.links.bus0),
            "bus1_retro" : list(retro.links.bus1),
            "x0_retro" : list(retro.links.bus0.map(df.x).round(coord_round)),
            "y0_retro" : list(retro.links.bus0.map(df.y).round(coord_round)),
            "x1_retro" : list(retro.links.bus1.map(df.x).round(coord_round)),
            "y1_retro" : list(retro.links.bus1.map(df.y).round(coord_round)),
            "p_nom_opt_retro" : list(retro.links.p_nom_opt.round(0)),
       }


   

In [ ]:
normal.buses[normal.buses.carrier == carrier].head()['ct'] = normal.buses[normal.buses.carrier == carrier].head().index.str[:2]

normal.buses[normal.buses.carrier == carrier].head()

In [ ]:
df.head()

In [ ]:

 #   ,df.loc[normal.lines.bus0[i]]['y']][df.loc[normal.lines.bus1[i]]['x'],df.loc[normal.lines.bus1[i]]['y']] ,data['y0_AC'][i] ][data['x1_AC'][i], data['y1_AC'][i]

In [ ]:
# for i in range(len(normal.lines.s_nom_opt)): 
#     if df.loc[normal.lines.bus1[i]]['x'].round(3) == [data['x1_AC'][i]]:
#         print( i, "ok")
#     else:
#         print("no")


In [ ]:
[data['x0_AC'][0],data['y0_AC'][0] ],[data['x1_AC'][0], data['y1_AC'][0]]

In [ ]:
fig = go.Figure(data=go.Choropleth(
    geojson=j_file,
    featureidkey= 'properties.COUNTRY_ID',
    locationmode='geojson-id',
    locations=df['ct'],
    z=df['v_nom'], #Platzhalter für später = Soll eventuell mal verbrauch werden 
    autocolorscale=False,
    showscale=False,
    hoverinfo = 'text',
    text = df['ct']
    
))

if carrier == 'AC':
    
    for i in range(len(data['x0_AC'])):
        fig.add_trace(
            go.Scattergeo(
                locationmode = 'geojson-id',
                lon = [data['x0_AC'][i], data['x1_AC'][i]],
                lat = [data['y0_AC'][i], data['y1_AC'][i]],
                mode = 'lines',
                line = dict(width = (data['s_nom_opt'][i])/1e4,color = 'green'),
                # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
            )
        )
else:
    
    for i in range(len(data['x0'])):
        fig.add_trace(
            go.Scattergeo(
                locationmode = 'geojson-id',
                lon = [data['x0'][i], data['x1'][i]],
                lat = [data['y0'][i], data['y1'][i]],
                mode = 'lines',
                line = dict(width = (data['p_nom_opt'][i])/1e4,color = 'red'),
                # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
            )
        )
    for i in range(len(data['x0_retro'])):
        fig.add_trace(
            go.Scattergeo(
                locationmode = 'geojson-id',
                lon = [data['x0_retro'][i], data['x1_retro'][i]],
                lat = [data['y0_retro'][i], data['y1_retro'][i]],
                mode = 'lines',
                line = dict(width = (data['p_nom_opt_retro'][i])/1e4,color = 'blue'),
                hoverinfo = 'text',
                text = data['p_nom_opt_retro'][i],
                # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
            )
        )


fig.update_geos(
    visible=False, resolution=50, scope="europe",
    showcountries=True, countrycolor="Black",
    #showsubunits=True, subunitcolor="Blue"
)



fig.update_layout(
    autosize=False,
    width=1000,
    height=1000,
    showlegend = False,
)
fig.show()

In [ ]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc 


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SOLAR])
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

In [ ]:
if __name__=='__main__':
    app.run_server(port=8041)


In [ ]:
import plotly.graph_objects as go
import pandas as pd

df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_airports.head()

df_flight_paths = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_aa_flight_paths.csv')
df_flight_paths.head()

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df_airports['long'],
    lat = df_airports['lat'],
    hoverinfo = 'text',
    text = df_airports['airport'],
    mode = 'markers',
    marker = dict(
        size = 2,
        color = 'rgb(255, 0, 0)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

flight_paths = []
for i in range(len(df_flight_paths)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df_flight_paths['start_lon'][i], df_flight_paths['end_lon'][i]],
            lat = [df_flight_paths['start_lat'][i], df_flight_paths['end_lat'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
        )
    )

fig.update_layout(
    title_text = 'Feb. 2011 American Airline flight paths<br>(Hover for airport names)',
    showlegend = False,
    geo = dict(
        scope = 'north america',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()

In [ ]:
[lon_x0,lon_x1]

In [ ]:
data['x0'][1], data['x1'][1]

In [ ]:
with open('json_data.json', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
with open('json_data.json') as geofile:
    link_file = json.load(geofile)

In [ ]:
df2 = df1[df1.iso_alpha == 'AGO']

In [ ]:
normal.links['end'] = normal.links.bus1.str[:2]

In [ ]:
df2 = df1[df1.continent == "Africa"][:2]

In [ ]:
df1

In [ ]:
n.links[n.links.carrier == 'helmeth'][['bus0','bus1','bus2','bus3','carrier']].head()

In [ ]:
n.links_t.p1[["AT0 0 Sabatier"]].head()

In [ ]:
n.links[n.links.index.str.contains('AT0 0 biogas')]

In [ ]:
n.links.carrier.unique()